# Home work 7

Завантаження файлу з оцінками користувачів

In [1]:
import pandas as pd

ratings_path = 'ml-latest-small/ratings.csv'
ratings_df = pd.read_csv(ratings_path)

ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Матрична факторизація за допомогою алгоритму SVD

In [2]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.accuracy import rmse

# Налаштування формату даних для Surprise
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# Розділення датасету на тренувальну та тестову вибірки
trainset, testset = train_test_split(data, test_size=0.25)

# Створення та тренування моделі SVD
model = SVD()
model.fit(trainset)

# Передбачення оцінок на тестовій вибірці
predictions = model.test(testset)

# Обчислення RMSE для оцінки якості моделі
rmse(predictions)


RMSE: 0.8784


0.878426519311619

Підбір найкращих параметрів  для алгоритму SVD за допомогою крос-валідації із застосуванням пошуку по сітці

In [3]:
from surprise.model_selection import GridSearchCV

# Визначення параметрів для тестування
param_grid = {
    'n_epochs': [5, 10, 20],  # Кількість епох
    'lr_all': [0.002, 0.005],  # Швидкість навчання
    'n_factors': [50, 100, 200],  # Кількість факторів
    'reg_all': [0.02, 0.05, 0.1]  # Коефіцієнт регуляризації
}

# Створення об'єкта GridSearch з SVD алгоритмом і визначеними параметрами
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

# Виконання пошуку з крос-валідацією
gs.fit(data)

# Найкращі знайдені параметри та їх RMSE
best_params_SVD = gs.best_params['rmse']
best_score_SVD = gs.best_score['rmse']

print(f"Best RMSE SVD: {best_score_SVD}")
print("Best parameters SVD:", best_params_SVD)

Best RMSE SVD: 0.8743944176100986
Best parameters SVD: {'n_epochs': 20, 'lr_all': 0.005, 'n_factors': 50, 'reg_all': 0.05}


Порівняння алгоритмів SVD, SVD++, NMF за допомогою крос-валідації

In [4]:
from surprise import SVDpp, NMF
from surprise.model_selection import cross_validate

# Створення об'єктів алгоритмів
algorithms = {
    'SVD': SVD(**best_params_SVD),  # використовуємо раніше визначені оптимальні параметри
    'SVD++': SVDpp(),               # використовуємо параметри за замовчуванням
    'NMF': NMF()                    # використовуємо параметри за замовчуванням
}

# Порівняння алгоритмів за допомогою крос-валідації
results = {}
for name, algorithm in algorithms.items():
    cv_results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=True)
    results[name] = cv_results['test_rmse'].mean()

# Виведення середнього RMSE для кожного алгоритму
for name, rmse in results.items():
    print(f"{name}: Mean RMSE = {rmse}")


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8678  0.8703  0.8759  0.8709  0.8632  0.8696  0.0042  
Fit time          0.66    0.68    0.70    0.67    0.72    0.69    0.02    
Test time         0.08    0.15    0.17    0.08    0.15    0.13    0.04    
Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8603  0.8558  0.8690  0.8645  0.8518  0.8603  0.0061  
Fit time          43.02   42.96   42.96   43.38   43.14   43.09   0.16    
Test time         8.82    8.98    8.84    8.86    9.07    8.91    0.09    
Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9235  0.9187  0.9150  0.9256  0.9252  0.9216  0.0041  
Fit time          1.95    1.98    2.01    1.97    1.97    1.98    0.02    
Test time         0.07    

**Висновок:** найкращу середню точність за критерієм RMSE показує алгоритм SVD++